In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\sumitsuman\\Documents\\Insurance_pre_authorization'

In [3]:
machine_learning_api_script_file = "./machine_learning_api.py"

In [4]:
from flask import Flask, abort, jsonify, request
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,accuracy_score,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report, average_precision_score 
from sklearn.pipeline import Pipeline

import pickle

# global graph,model
# graph = tf.get_default_graph()


class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                if output[colname].dtype == 'object':
                    print("Encoding "+colname)
                    output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
encoding_pipeline = Pipeline([
('encoding',MultiColumnLabelEncoder())
# add more pipeline steps as needed
])

app = Flask(__name__)
# Load Model
label_encoder_model = pickle.load(open('encoding_model.sav', 'rb'))
classification_model = pickle.load(open('gradientboost.sav', 'rb'))

columns=[u'INDEMNITY_PAID_TO_DATE_USD_AM', u'EXPENSE_PAID_TO_DATE_USD', u'TYPE_NM',
       u'FATAL_INJURY_IN', u'HAS_INJURY_DETAILS', u'INJURY_BODY_PART_DS',
       u'INJURY_CATEGORY_DS', u'INJURY_DS', u'INJURY_SEVERITY_DS',
       u'MAIN_CAUSE_DS', u'SUB_CAUSE_DS', u'INSURED_NM', u'CLAIM_TYPE',
       u'SUBLINE_OF_BUSINESS_DS', u'SUBLINE_OF_BUSINESS_CD',
       u'LINE_OF_BUSINESS_CD', u'LINE_OF_BUSINESS_DS', u'Fraud',
       u'Duration_of_claim_after_policy_effective',
       u'Duration_left_to_policy_expiry']


@app.route('/make_prediction', methods=['POST'])
def make_prediction():
    # read json object and conver to json string
    data = json.dumps(request.get_json(force=True))
    # create pandas dataframe using json string
    df = pd.read_json(data)
    cc=df[columns]
    df2=label_encoder_model.fit_transform(cc)
    X_test=df2.drop(columns = ["Fraud"],axis=1)
    y_test=df2["Fraud"]
    

    
#     with graph.as_default():
    model_predict_ = classification_model.predict(X_test.values)
    model_accuracy_ = accuracy_score(y_test,model_predict_)
    roc_auc_score_model_ = roc_auc_score(y_test,model_predict_)
    # create response dataframe
#     yes=pd.DataFrame.from_records(y)
#     aa=pd.DataFrame()
    cc['predicted']=model_predict_
#     aa['Predicted']=yes[0]
#     aa['Actual']=actuals
    return cc.to_json()
  

 

if __name__ == '__main__':
    app.run(port=10001,debug=False)
    # host flask app at port 10001

 * Running on http://127.0.0.1:10001/ (Press CTRL+C to quit)
C:\Users\sumitsuman\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
127.0.0.1 - - [01/Apr/2019 17:33:24] "POST /make_prediction HTTP/1.1" 200 -


Encoding TYPE_NM
Encoding FATAL_INJURY_IN
Encoding INJURY_BODY_PART_DS
Encoding INJURY_CATEGORY_DS
Encoding INJURY_DS
Encoding INJURY_SEVERITY_DS
Encoding MAIN_CAUSE_DS
Encoding SUB_CAUSE_DS
Encoding INSURED_NM
Encoding CLAIM_TYPE
Encoding SUBLINE_OF_BUSINESS_DS
Encoding LINE_OF_BUSINESS_DS
Encoding Fraud


C:\Users\sumitsuman\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
127.0.0.1 - - [01/Apr/2019 17:38:28] "POST /make_prediction HTTP/1.1" 200 -


Encoding TYPE_NM
Encoding FATAL_INJURY_IN
Encoding INJURY_BODY_PART_DS
Encoding INJURY_CATEGORY_DS
Encoding INJURY_DS
Encoding INJURY_SEVERITY_DS
Encoding MAIN_CAUSE_DS
Encoding SUB_CAUSE_DS
Encoding INSURED_NM
Encoding CLAIM_TYPE
Encoding SUBLINE_OF_BUSINESS_DS
Encoding LINE_OF_BUSINESS_DS
Encoding Fraud
